### CALL CENTER OPERATIONS ANALYTICS
Author Diana Kung'u

Questions:


In [ ]:
# Load libraries
import pandas as pd
import time
import matplotlib

In [27]:

#Read data
#.............................................................................................
chunks = pd.read_csv(r'.\Data\call center.csv', parse_dates=['Call Date Time'], chunksize= 5000)
call_df = pd.concat(chunks, ignore_index= True)

In [38]:
df = call_df.copy()
df.head(3)

,Abandon Flag,Agent,Agent Selection Group,Call #,Call Date Time,Call Reason,Customer,First Contact Resolution Flag,Talk Time (s),Wait Time (s)
0,N,Anabel Grebert,Other,3800023,2021-11-30 09:46:00,Billing Questions,"Anderson, Schoen and Pfannerstill",N,143,5
1,N,Anabel Grebert,Other,3796097,2021-11-29 08:21:00,Billing Questions,"Anderson, Schoen and Pfannerstill",N,38,6
2,N,Anabel Grebert,Other,3772171,2021-11-18 10:20:00,Billing Questions,"Anderson, Schoen and Pfannerstill",N,54,6


In [46]:
#Data Processing 
#Split call date and time fields
Call_Date, Call_Time= zip(*[(d.date(), d.time()) for d in df['Call Date Time']])
df['Call Date'], df['Call Time'] =[Call_Date, Call_Time]

#Format Date and time columns to datetime objects
df['Call Date'] = pd.to_datetime(df['Call Date'], format='%Y-%m-%d')
df['Call Time'] = df['Call Time'].apply(lambda x: x.strftime('%H:%M'))
df.head(2)

,Abandon Flag,Agent,Agent Selection Group,Call #,Call Date Time,Call Reason,Customer,First Contact Resolution Flag,Talk Time (s),Wait Time (s),Call Date,Call Time
0,N,Anabel Grebert,Other,3800023,2021-11-30 09:46:00,Billing Questions,"Anderson, Schoen and Pfannerstill",N,143,5,2021-11-30,09:46:00
1,N,Anabel Grebert,Other,3796097,2021-11-29 08:21:00,Billing Questions,"Anderson, Schoen and Pfannerstill",N,38,6,2021-11-29,08:21:00


In [54]:
# Parse the Year, Month Day, and hour of each call
df['Year'] = df['Call Date'].dt.year
df['Month'] = df['Call Date'].dt.month
df['Day of Week'] = [d.strftime("%A") for d in df['Call Date']]
df['Hour'] =  df['Call Date Time'].dt.hour
df.head(3)


,Abandon Flag,Agent,Agent Selection Group,Call #,Call Date Time,Call Reason,Customer,First Contact Resolution Flag,Talk Time (s),Wait Time (s),Call Date,Call Time,Month,Day of Week,Year,Hour
0,N,Anabel Grebert,Other,3800023,2021-11-30 09:46:00,Billing Questions,"Anderson, Schoen and Pfannerstill",N,143,5,2021-11-30,09:46,11,Tuesday,2021,9
1,N,Anabel Grebert,Other,3796097,2021-11-29 08:21:00,Billing Questions,"Anderson, Schoen and Pfannerstill",N,38,6,2021-11-29,08:21,11,Monday,2021,8
2,N,Anabel Grebert,Other,3772171,2021-11-18 10:20:00,Billing Questions,"Anderson, Schoen and Pfannerstill",N,54,6,2021-11-18,10:20,11,Thursday,2021,10
3,N,Benjie Pyott,Other,3827739,2021-12-09 11:35:00,Billing Questions,"Anderson, Schoen and Pfannerstill",N,633,45,2021-12-09,11:35,12,Thursday,2021,11
4,N,Berton Flecknoe,Other,3181834,2021-04-09 11:09:00,Billing Questions,"Anderson, Schoen and Pfannerstill",N,39,7,2021-04-09,11:09,4,Friday,2021,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251359,N,Margarete Von Der Empten,Other,3792016,2021-11-25 16:50:00,Technical Issues,West-McLaughlin,Y,123,121,2021-11-25,16:50,11,Thursday,2021,16
251360,N,Margarete Von Der Empten,Other,3098579,2021-03-03 13:33:00,Technical Issues,West-McLaughlin,Y,265,121,2021-03-03,13:33,3,Wednesday,2021,13
251361,N,Margarete Von Der Empten,Other,3029719,2021-01-28 09:43:00,Technical Issues,West-McLaughlin,Y,182,122,2021-01-28,09:43,1,Thursday,2021,9
251362,N,Margarete Von Der Empten,Other,3635876,2021-10-01 11:30:00,Technical Issues,West-McLaughlin,Y,172,123,2021-10-01,11:30,10,Friday,2021,11


#### Call Reasons
1. Frequency of call reasons

In [82]:
# Unique call reasons
reasons  = df['Call Reason'].unique().tolist()
#Frequency of each reason
var = df.groupby(['Call Reason']).agg(No_Calls =('Call #', 'count'))
var['Proportion'] =var['No_Calls']*100/var['No_Calls'].sum()
var['Proportion'] = var['Proportion'].map('{:,.1f}%'.format)
var

,No_Calls,Proportion
Call Reason,,
Billing Questions,55192,22.0%
Internet Outage,72671,28.9%
New Customer,5871,2.3%
Other,927,0.4%
Product/Service Problems,21905,8.7%
Technical Issues,94798,37.7%
